# **Prompt Generation**

In this notebook is illustrated how to generate prompts by using the implemented libraries.

Note that these prompts are essentially designed for GPT models, or for those supporting the following roles:
- **system**,  which provides high-level instructions to guide model’s behavior throughout the conversation;
- **user**, it is the model’s response, usually simulated when using few-shot prompting;
- **assistant**, that presents queries related to the task the LLM is asked to perform.

In [1]:
import os
import sys
sys.path.append('\\'.join(os.getcwd().split('\\')[:-1])+'\\src')

import src.data.TFQuestionnairesDataset as tf_qst
import src.prompts.PromptGenerator as pr

from pprint import pprint

## **Questionnaire generation**

The first step of this thesis work is the questionnaire generaion. The designed prompts are reported below.

In [2]:
# ----------------
# Load data
# ----------------
dataset = tf_qst.TFQuestionnairesDataset()
dataset.load_data()

samples = dataset.get_sample_questionnaire_data()
question_types = samples.get_question_types()

### *System prompt*

The **system** prompt has two variants according to the given specifications:
1. with all parameters: topic, number and type of questions to be generated
2. with only the questionnaire topic

In [3]:
system_generator = pr.PromptGenerator("system")

with_all_params = system_generator.generate_prompt(has_full_params=True, topic=samples.questionnaires["NAME"], question_type="Single choice", question_nuber=5, 
                                                   question_types_data=question_types)

with_only_topic = system_generator.generate_prompt(has_full_params=False, topic=samples.questionnaires["NAME"], question_types_data=question_types)

Both of them are structured as follows:
1. **Role definition**, used to define the role the LLM has to impersonate;
2. **Input definition**, which define the information that the user can specify and (eventually) the used format;
3. **Question types**, used to define the admissible question types;
4. **Task definition**, which is used to instruct the LLM on the task it has to perform;
5. **Output definition**, it constrates the LLM to use the specified format while generating its response.

In [4]:
pprint(with_all_params)

('You are a Questionnaire Generator and you work in the Human Resources '
 'Management field. The user will ask you generate surveys mostly.The user has '
 'to specify the following elements in the request:\n'
 '    - topic\n'
 '    - number of questions\n'
 '    - type of questions\n'
 '\n'
 "The admited question's types are the following:\n"
 '  - ID: 1, CODE: SingleChoice, NAME: Single choice, DESCRIPTION: Use this '
 'type of question to choose one answer from a list. \n'
 '  - ID: 2, CODE: MultiChoice, NAME: Multi choice, DESCRIPTION: Use this type '
 'of question to choose one or more answer from a list. \n'
 '  - ID: 3, CODE: RatingScale, NAME: Rating scale, DESCRIPTION: Use this type '
 'of question to rate something. \n'
 '  - ID: 4, CODE: Comment, NAME: Comment, DESCRIPTION: Use this type of '
 'question to aquire feedback. \n'
 '  - ID: 5, CODE: ReorderItems, NAME: Reorder items, DESCRIPTION: Use this '
 'type of question to reorder items. \n'
 '  - ID: 6, CODE: SumDistribut

In [5]:
pprint(with_only_topic)

('You are a Questionnaire Generator and you work in the Human Resources '
 'Management field. The user will ask you generate surveys mostly.The user has '
 "to specify the questionnaire's topic.\n"
 'Decide the proper numer and type of questions is up to you.\n'
 "The admited question's types are the following:\n"
 '  - ID: 1, CODE: SingleChoice, NAME: Single choice, DESCRIPTION: Use this '
 'type of question to choose one answer from a list. \n'
 '  - ID: 2, CODE: MultiChoice, NAME: Multi choice, DESCRIPTION: Use this type '
 'of question to choose one or more answer from a list. \n'
 '  - ID: 3, CODE: RatingScale, NAME: Rating scale, DESCRIPTION: Use this type '
 'of question to rate something. \n'
 '  - ID: 4, CODE: Comment, NAME: Comment, DESCRIPTION: Use this type of '
 'question to aquire feedback. \n'
 '  - ID: 5, CODE: ReorderItems, NAME: Reorder items, DESCRIPTION: Use this '
 'type of question to reorder items. \n'
 '  - ID: 6, CODE: SumDistribution, NAME: Sum distribution, D

### *Assistant prompt*

The **assistant** prompt represents the LLM's output. It's used only when using the *few-shot* prompting technique.

In [6]:
assistant_generator = pr.PromptGenerator("assistant")

response = assistant_generator.generate_prompt(json="TO_BE_PLACED")
pprint(response)

'Questionnaire: \nTO_BE_PLACED\n'


### *User prompt*

The **user** prompt reprents the user request. Similarly to the *system* prompt, it has two variants.

In [7]:
user_generator = pr.PromptGenerator("user")

with_all_params = user_generator.generate_prompt(has_full_params=True, topic=samples.questionnaires["NAME"], question_type="Single choice", question_nuber=5)
with_only_topic = user_generator.generate_prompt(has_full_params=False, topic=samples.questionnaires["NAME"])

In [8]:
pprint(with_all_params)

'Generate me a questionnaire on Stress Survey with Single choice 5 questions'


In [9]:
pprint(with_only_topic)

('Generate me a questionnaire on Stress Survey with Single choice 5 '
 'questionsGenerate me a questionnaire on Stress Survey')
